In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = torchvision.datasets.MNIST(root='../../data', train=True, transform=transforms.ToTensor(), download=False)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Recurrent Neural Network
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(in_features=hidden_size, out_features=num_classes)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0)) # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state fo the last time step
        out = self.fc(out[:, -1,:])
        return out
    


In [6]:
# Net, Loss and optimizer
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.5473
Epoch [1/2], Step [200/600], Loss: 0.3761
Epoch [1/2], Step [300/600], Loss: 0.1853
Epoch [1/2], Step [400/600], Loss: 0.2628
Epoch [1/2], Step [500/600], Loss: 0.0914
Epoch [1/2], Step [600/600], Loss: 0.0369
Epoch [2/2], Step [100/600], Loss: 0.1233
Epoch [2/2], Step [200/600], Loss: 0.1414
Epoch [2/2], Step [300/600], Loss: 0.2027
Epoch [2/2], Step [400/600], Loss: 0.0486
Epoch [2/2], Step [500/600], Loss: 0.1076
Epoch [2/2], Step [600/600], Loss: 0.0979


In [8]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    total_step = len(test_loader)
    for i,(images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if (i+1)% 30 == 0:
            print('[{}/{}]Test Accuracy of the model on the 10000 test images: {:.4f} %'.format(i+1, total_step, 100 * correct / total)) 

[30/600]Test Accuracy of the model on the 10000 test images: 97.5333 %
[60/600]Test Accuracy of the model on the 10000 test images: 97.6167 %
[90/600]Test Accuracy of the model on the 10000 test images: 97.5667 %
[120/600]Test Accuracy of the model on the 10000 test images: 97.5500 %
[150/600]Test Accuracy of the model on the 10000 test images: 97.4533 %
[180/600]Test Accuracy of the model on the 10000 test images: 97.4278 %
[210/600]Test Accuracy of the model on the 10000 test images: 97.4238 %
[240/600]Test Accuracy of the model on the 10000 test images: 97.4875 %
[270/600]Test Accuracy of the model on the 10000 test images: 97.5296 %
[300/600]Test Accuracy of the model on the 10000 test images: 97.5133 %
[330/600]Test Accuracy of the model on the 10000 test images: 97.4818 %
[360/600]Test Accuracy of the model on the 10000 test images: 97.5028 %
[390/600]Test Accuracy of the model on the 10000 test images: 97.4949 %
[420/600]Test Accuracy of the model on the 10000 test images: 97.48

In [9]:
torch.save(model.state_dict(), "model.ckpt")